#CODE for DPGAN
## Refrences: 

https://blog.tensorflow.org/2019/03/introducing-tensorflow-privacy-learning.html

http://www.cleverhans.io/privacy/2019/03/26/machine-learning-with-differential-privacy-in-tensorflow.html

https://medium.com/gretel-ai/using-generative-differentially-private-models-to-build-privacy-enhancing-synthetic-datasets-c0633856184


https://github.com/ricardocarvalhods/dpcgan/blob/master/DP_CGAN_MNIST.ipynb

https://colab.research.google.com/github/ricardocarvalhods/dpcgan/blob/master/DP_CGAN_MNIST.ipynb





# Loading the Credit Cart Fraud Detection
(results are not included for this dataset) 
** please do not run this cell

In [ ]:
from pandas import read_csv
dataframe = read_csv('/content/drive/MyDrive/Privacy/creditcard.csv', header=None)


# load the dataset
def load_dataset(full_path = '/content/drive/MyDrive/Privacy/creditcard.csv'):
	# load the dataset as a numpy array
	data = read_csv(full_path, header=None)
	# retrieve numpy array
	data = data.values
	# split into input and output elements
	X, y = data[:, :-1], data[:, -1]
	return X, y

X, y = load_dataset()
print(X[1:2],y[1:2])

import numpy as np
train_labels = y.reshape((284807, 1))
COND_num_classes = 2 # Number of classes
train_labels_vec = np.zeros((len(train_labels), COND_num_classes), dtype='float32')
for i, label in enumerate(train_labels):
    train_labels_vec[i, int(train_labels[i])] = 1.0

train_data = X.astype('float32')
print(train_data.shape,train_labels_vec.shape)

# Loading the train real dataset (Banknote Authentication Dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''

#CODE for DPGAN
## Refrences: 

https://blog.tensorflow.org/2019/03/introducing-tensorflow-privacy-learning.html

http://www.cleverhans.io/privacy/2019/03/26/machine-learning-with-differential-privacy-in-tensorflow.html

https://medium.com/gretel-ai/using-generative-differentially-private-models-to-build-privacy-enhancing-synthetic-datasets-c0633856184


https://github.com/ricardocarvalhods/dpcgan/blob/master/DP_CGAN_MNIST.ipynb

https://colab.research.google.com/github/ricardocarvalhods/dpcgan/blob/master/DP_CGAN_MNIST.ipynb
'''


# load the dataset
from pandas import read_csv
from scipy.special import expit
from sklearn.utils import shuffle
import pickle
def load_dataset(full_path = '/content/drive/MyDrive/Privacy/real_data_train.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  print(data)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X, y = load_dataset()
X = expit(X)
X, y = shuffle(X, y)
print(X.shape,y.shape)
print(X[0])

            0        1         2        3  4
131   5.13020   8.6703  -2.89130 -1.50860  0
610  -0.23610   9.3221   2.13070 -4.37930  0
1314 -3.50600 -12.5667  15.16060 -0.75216  1
990  -4.57700   3.4515   0.66719 -0.94742  1
292   4.13490   6.1189  -2.42940 -0.19613  0
...       ...      ...       ...      ... ..
717   2.98560   7.2673  -0.40900 -2.24310  0
87    0.87603   6.8141   0.84198 -0.17156  0
986   0.84546   3.4826  -3.63070 -1.39610  1
1027  0.11686   3.7350  -4.43790 -4.37410  1
1013 -0.50816   2.8680  -1.81080 -2.26120  1

[1000 rows x 5 columns]
(1000, 4) (1000,)
[0.19190177 0.19278725 0.89229492 0.60364356]


In [ ]:
import numpy as np

train_labels = y.reshape((1000, 1))
COND_num_classes = 2 # Number of classes
train_labels_vec = np.zeros((len(train_labels), COND_num_classes), dtype='float32')
for i, label in enumerate(train_labels):
    train_labels_vec[i, int(train_labels[i])] = 1.0

train_data = X.astype('float32')
print(train_data.shape,train_labels_vec.shape)

(1000, 4) (1000, 2)


In [ ]:
!pip install tensorflow_privacy
!pip install sklearn

In [ ]:
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K
import time

from IPython import display

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer


from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import roc_curve, auc
ConvergenceWarning('ignore')

sklearn.exceptions.ConvergenceWarning('ignore')

# Model and optimizer setting

In [ ]:
from absl import logging
import collections

from tensorflow_privacy.privacy.analysis import privacy_ledger
from tensorflow_privacy.privacy.dp_query import gaussian_query

def make_optimizer_class(cls):
  """Constructs a DP optimizer class from an existing one."""
  parent_code = tf.compat.v1.train.Optimizer.compute_gradients.__code__
  child_code = cls.compute_gradients.__code__
  GATE_OP = tf.compat.v1.train.Optimizer.GATE_OP  # pylint: disable=invalid-name
  if child_code is not parent_code:
    logging.warning(
        'WARNING: Calling make_optimizer_class() on class %s that overrides '
        'method compute_gradients(). Check to ensure that '
        'make_optimizer_class() does not interfere with overridden version.',
        cls.__name__)

  class DPOptimizerClass(cls):
    """Differentially private subclass of given class cls."""

    _GlobalState = collections.namedtuple(
      '_GlobalState', ['l2_norm_clip', 'stddev'])
    
    def __init__(
        self,
        dp_sum_query,
        num_microbatches=None,
        unroll_microbatches=False,
        *args,  # pylint: disable=keyword-arg-before-vararg, g-doc-args
        **kwargs):
      """Initialize the DPOptimizerClass.

      Args:
        dp_sum_query: DPQuery object, specifying differential privacy
          mechanism to use.
        num_microbatches: How many microbatches into which the minibatch is
          split. If None, will default to the size of the minibatch, and
          per-example gradients will be computed.
        unroll_microbatches: If true, processes microbatches within a Python
          loop instead of a tf.while_loop. Can be used if using a tf.while_loop
          raises an exception.
      """
      super(DPOptimizerClass, self).__init__(*args, **kwargs)
      self._dp_sum_query = dp_sum_query
      self._num_microbatches = num_microbatches
      self._global_state = self._dp_sum_query.initial_global_state()

      self._unroll_microbatches = unroll_microbatches

    def compute_gradients(self,
                          loss,
                          var_list,
                          gate_gradients=GATE_OP,
                          aggregation_method=None,
                          colocate_gradients_with_ops=False,
                          grad_loss=None,
                          gradient_tape=None,
                          curr_noise_mult=0,
                          curr_norm_clip=1):

      self._dp_sum_query = gaussian_query.GaussianSumQuery(curr_norm_clip, 
                                                           curr_norm_clip*curr_noise_mult)
      self._global_state = self._dp_sum_query.make_global_state(curr_norm_clip, 
                                                                curr_norm_clip*curr_noise_mult)
      

      # TF is running in Eager mode, check we received a vanilla tape.
      if not gradient_tape:
        raise ValueError('When in Eager mode, a tape needs to be passed.')

      vector_loss = loss()
      if self._num_microbatches is None:
        self._num_microbatches = tf.shape(input=vector_loss)[0]
      sample_state = self._dp_sum_query.initial_sample_state(var_list)
      microbatches_losses = tf.reshape(vector_loss, [self._num_microbatches, -1])
      sample_params = (self._dp_sum_query.derive_sample_params(self._global_state))

      def process_microbatch(i, sample_state):
        """Process one microbatch (record) with privacy helper."""
        microbatch_loss = tf.reduce_mean(input_tensor=tf.gather(microbatches_losses, [i]))
        grads = gradient_tape.gradient(microbatch_loss, var_list)
        sample_state = self._dp_sum_query.accumulate_record(sample_params, sample_state, grads)
        return sample_state
    
      for idx in range(self._num_microbatches):
        sample_state = process_microbatch(idx, sample_state)

      if curr_noise_mult > 0:
        grad_sums, self._global_state = (self._dp_sum_query.get_noised_result(sample_state, self._global_state))
      else:
        grad_sums = sample_state

      def normalize(v):
        return v / tf.cast(self._num_microbatches, tf.float32)

      final_grads = tf.nest.map_structure(normalize, grad_sums)
      grads_and_vars = final_grads#list(zip(final_grads, var_list))
    
      return grads_and_vars

  return DPOptimizerClass


def make_gaussian_optimizer_class(cls):
  """Constructs a DP optimizer with Gaussian averaging of updates."""

  class DPGaussianOptimizerClass(make_optimizer_class(cls)):
    """DP subclass of given class cls using Gaussian averaging."""

    def __init__(
        self,
        l2_norm_clip,
        noise_multiplier,
        num_microbatches=None,
        ledger=None,
        unroll_microbatches=False,
        *args,  # pylint: disable=keyword-arg-before-vararg
        **kwargs):
      dp_sum_query = gaussian_query.GaussianSumQuery(
          l2_norm_clip, l2_norm_clip * noise_multiplier)

      if ledger:
        dp_sum_query = privacy_ledger.QueryWithLedger(dp_sum_query,
                                                      ledger=ledger)

      super(DPGaussianOptimizerClass, self).__init__(
          dp_sum_query,
          num_microbatches,
          unroll_microbatches,
          *args,
          **kwargs)

    @property
    def ledger(self):
      return self._dp_sum_query.ledger

  return DPGaussianOptimizerClass

In [ ]:
GradientDescentOptimizer = tf.compat.v1.train.GradientDescentOptimizer
DPGradientDescentGaussianOptimizer_NEW = make_gaussian_optimizer_class(GradientDescentOptimizer)

In [ ]:
# Edit to define folder
result_dir = '/content/drive/MyDrive/Privacy'

# DO NOT EDIT THIS CELL

# Define checkpoint dir and prefix
checkpoint_dir = result_dir + '/training_checkpoints_NEW_DPGAN'

def checkpoint_name(title):  
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt__" + title)
  return(checkpoint_prefix)

In [ ]:
# Dimension of Latent Space - Does NOT affect DP-EPSILON
Z_DIM = 4
#noisy nput vector

However, to implement WGAN, we should notice few things from the above:

No 
log
 in the loss. The output of 
D
 is no longer a probability, hence we do not apply sigmoid at the output of 
D
Clip the weight of 
D
Train 
D
 more than 
G
Use RMSProp instead of ADAM
Lower learning rate, the paper uses  
α
=
0.00005

In [ ]:
#define the conditional GAN
def make_generator_model_FCC():
    # INPUT: label input
    # in_label = layers.Input(shape=(COND_num_classes,))

    # INPUT: data generator input
    in_lat = layers.Input(shape=(Z_DIM,))

    # MERGE
    # merge = layers.concatenate([in_lat, in_label], axis=1)

    ge1 = layers.Dense(20, use_bias=True)(in_lat)
    ge1 = layers.ReLU()(ge1)


    ge3 = layers.Dense(4, use_bias=True, activation="tanh")(ge1)
    out_layer = layers.Reshape((4,))(ge3)

    model = models.Model(in_lat, out_layer)

    return model

def make_discriminator_model_FCC():
    # INPUT: Label
    # in_label = layers.Input(shape=(COND_num_classes,))

    # INPUT: data
    in_data = layers.Input(shape=(4,))
    # in_data_b = layers.Flatten()(in_data)

    # MERGE
    # merge = layers.concatenate([in_data, in_label], axis=1)

    di1 = layers.Dense(20, use_bias=True)(in_data)
    di1 = layers.ReLU()(di1)

    di2 = layers.Dense(4, use_bias=True)(di1)
    di2 = layers.ReLU()(di2)

    out_layer = layers.Dense(1, use_bias=True)(di2)

    model = models.Model(in_data, out_layer)

    return model

In [ ]:
generator = make_generator_model_FCC()
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 20)                100       
_________________________________________________________________
re_lu (ReLU)                 (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 84        
_________________________________________________________________
reshape (Reshape)            (None, 4)                 0         
Total params: 184
Trainable params: 184
Non-trainable params: 0
_________________________________________________________________


In [ ]:
discriminator = make_discriminator_model_FCC()
discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                100       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 84        
_________________________________________________________________
re_lu_2 (ReLU)               (None, 4)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 189
Trainable params: 189
Non-trainable params: 0
_____________________________________________________________

In [ ]:
# Test GEN created
noise = tf.Variable(tf.random.normal([1, Z_DIM]))
# noise_label = tf.Variable(np.array([0,1], dtype='float32').reshape((1,2)))
print(noise.shape)
# print(noise_label.shape)
generated_data = generator(noise, training=False)
print(generated_data)

decision = discriminator(generated_data)
print("decision",decision)

(1, 4)
tf.Tensor([[ 0.03868612 -0.00282908 -0.00720142 -0.22596738]], shape=(1, 4), dtype=float32)
decision tf.Tensor([[0.12446492]], shape=(1, 1), dtype=float32)


# Update


In [ ]:
cross_entropy_DISC = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
cross_entropy_GEN = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Notice the use of `tf.function`: This annotation causes the function to be "compiled".
@tf.function
def train_step_DISC(datas, noise):    
    with tf.GradientTape(persistent=True) as disc_tape_real:
        # This dummy call is needed to obtain the var list.
        dummy = discriminator(datas, training=True)
        var_list = discriminator.trainable_variables
        
        # In Eager mode, the optimizer takes a function that returns the loss.
        def loss_fn_real():
            real_output = discriminator(datas, training=True)
            disc_real_loss = cross_entropy_DISC(tf.ones_like(real_output), real_output)
            return disc_real_loss
        
        grads_and_vars_real = discriminator_optimizer.compute_gradients(loss_fn_real, 
                                                                        var_list, 
                                                                        gradient_tape=disc_tape_real, 
                                                                        curr_noise_mult=NOISE_MULT,
                                                                        curr_norm_clip=NORM_CLIP)
        
        # In Eager mode, the optimizer takes a function that returns the loss.
        def loss_fn_fake():
            generated_datas = generator(noise, training=True)
            fake_output = discriminator(generated_datas, training=True)
            disc_fake_loss = cross_entropy_DISC(tf.zeros_like(fake_output), fake_output)
            return disc_fake_loss
        
        grads_and_vars_fake = discriminator_optimizer.compute_gradients(loss_fn_fake,
                                                                        var_list, 
                                                                        gradient_tape=disc_tape_real,
                                                                        curr_noise_mult=0,
                                                                        curr_norm_clip=NORM_CLIP)
        disc_loss_r = loss_fn_real()
        disc_loss_f = loss_fn_fake()
        
        s_grads_and_vars = [(grads_and_vars_real[idx] + grads_and_vars_fake[idx])
                            for idx in range(len(grads_and_vars_real))]
        sanitized_grads_and_vars = list(zip(s_grads_and_vars, var_list))
        
        discriminator_optimizer.apply_gradients(sanitized_grads_and_vars)

    return(disc_loss_r, disc_loss_f)

# Notice the use of `tf.function`: This annotation causes the function to be "compiled".
@tf.function
def train_step_GEN(noise):
    with tf.GradientTape() as gen_tape:
        generated_datas = generator(noise, training=True)
        fake_output = discriminator(generated_datas, training=True)
        # if the generator is performing well, the discriminator will classify the fake datas as real (or 1)
        gen_loss = cross_entropy_GEN(tf.ones_like(fake_output), fake_output)
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    return(gen_loss)

#Train

In [ ]:
def train(dataset, title, verbose):
    for epoch in range(EPOCHS):
        start = time.time()

        i_gen = 0
        for data_batch in dataset:
            if verbose:
                print("Iteration: " + str(i_gen+1))
            
            noise = tf.random.normal([BATCH_SIZE, Z_DIM])
            # labels_to_gen = _random_choice(labels_gen_vec, BATCH_SIZE)
    
            d_loss_r, d_loss_f = train_step_DISC(data_batch, noise)
            if verbose:
                print("Loss DISC Real: " + str(tf.reduce_mean(d_loss_r)))
                print("Loss DISC Fake: " + str(tf.reduce_mean(d_loss_f)))

            if (i_gen + 1) % N_DISC == 0:
                g_loss_f = train_step_GEN(noise)
                if verbose:
                    print("Loss GEN Fake:: " + str(g_loss_f))

            i_gen = i_gen + 1

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        print(epoch)
        print("end of epoch")

        # Save the model
        # checkpoint.save("/content/drive/MyDrive/Privacy/training_checkpoints/positive_model" + str(epoch))
        checkpoint.save(file_prefix = checkpoint_name(title + "__epoch=" + str(epoch) + "__"))


        

# parameters setting

In [ ]:
BUFFER_SIZE = 1000 # Total size of training data
BATCH_SIZE = 10
NR_MICROBATCHES = 1 # Each batch of data is split in smaller units called microbatches.


NORM_CLIP = 1.1 # Does NOT affect EPSILON, but increases NOISE on gradients
NOISE_MULT = 1.15


DP_DELTA = 1e-5 # Needs to be smaller than 1/BUFFER_SIZE
EPOCHS = 1000


N_DISC = 1 # Number of times we train DISC before training GEN once


# Learning Rate for DISCRIMINATOR
LR_DISC = tf.compat.v1.train.polynomial_decay(learning_rate=0.150,
                                              global_step=tf.compat.v1.train.get_or_create_global_step(),
                                              decay_steps=10000,
                                              end_learning_rate=0.052,
                                              power=1)

if BATCH_SIZE % NR_MICROBATCHES != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [ ]:
# Obtain DP_EPSILON
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n = BUFFER_SIZE, 
                                              batch_size = BATCH_SIZE, 
                                              noise_multiplier = NOISE_MULT, 
                                              epochs = EPOCHS, 
                                              delta = DP_DELTA)

DP-SGD with sampling rate = 1% and noise_multiplier = 1.15 iterated over 100000 steps satisfies differential privacy with eps = 20.7 and delta = 1e-05.
The optimal RDP order is 2.25.


(20.734201516946577, 2.25)

In [ ]:
# SD of noise that will be added to gradients: sanity check
NOISE_MULT*NORM_CLIP

1.265

#Optimizer

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam()

discriminator_optimizer = DPGradientDescentGaussianOptimizer_NEW(
   learning_rate = LR_DISC,
   l2_norm_clip = NORM_CLIP,
   noise_multiplier = NOISE_MULT,
   num_microbatches = NR_MICROBATCHES)

# Training process


In [ ]:
# Create/reinitiate models
generator = make_generator_model_FCC()
discriminator = make_discriminator_model_FCC()

In [ ]:
# Create checkpoint structure
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
tf.random.set_seed(1)

# Batch and random shuffle training data
train_dataset = tf.data.Dataset.from_tensor_slices(train_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Fix some seeds to help visualize progress
seed = tf.random.normal([2, Z_DIM])


# GAN for negative

In [ ]:
negative =[]
positive = []

for k in range(0,len(y)):
  if y[k]==0.0:
    negative.append(X[k])
  else:
    positive.append(X[k])


In [ ]:
negative = np.array(negative)
print(negative.shape)   
print(negative[1])

(561, 4)
[0.97104697 0.94820612 0.74228903 0.60054585]


# train GAN for negative

In [ ]:
tf.random.set_seed(100)

# Batch and random shuffle training data
train_dataset_neg = tf.data.Dataset.from_tensor_slices(negative).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [ ]:
# train(train_dataset_neg, "neg", False)

## restore model , generate and save data


In [ ]:
title = "neg"
epoch_ckp = EPOCHS - 1

In [ ]:
# DO NOT EDIT
checkpoint_name = checkpoint_dir + "/ckpt__" + str(title) + "__epoch=" + str(19) + "__-" + str(20)

# RESTORE CHECKPOINT to variables defined on it
checkpoint.restore(checkpoint_name)

In [ ]:
# Number of neg data to generate
N_neg = 500

In [ ]:
Z_DIM=4
tf.random.set_seed(0)
noise_GEN = tf.random.normal([N_neg, Z_DIM])
# labels_GEN = set for CGAN
data_neg_GEN = generator(noise_GEN, training=False)

In [ ]:
import numpy
proto_tensor = tf.make_tensor_proto(data_neg_GEN)  # convert `tensor a` to a proto tensor
data = tf.make_ndarray(proto_tensor) #convert tf tesor to np array
print(data.shape)


In [ ]:
import pandas as pd 
pd.DataFrame(data).to_csv("/content/drive/MyDrive/Privacy/DPGAN_neg_NEWdata.csv")
# numpy.savetxt("/content/drive/MyDrive/Privacy/DPGAN_neg_data.csv", data, delimiter=",")

# repeat for positive data

In [ ]:
positive = np.array(positive)
print(positive.shape)

(439, 4)


In [ ]:
tf.random.set_seed(0)

# Batch and random shuffle training data
train_dataset_pos = tf.data.Dataset.from_tensor_slices(positive).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [ ]:
train(train_dataset_pos, "positive_data", False)

Time for epoch 1 is 2.0329301357269287 sec
0
end of epoch
Time for epoch 2 is 0.07017159461975098 sec
1
end of epoch
Time for epoch 3 is 0.06985855102539062 sec
2
end of epoch
Time for epoch 4 is 0.07195663452148438 sec
3
end of epoch
Time for epoch 5 is 0.0706939697265625 sec
4
end of epoch
Time for epoch 6 is 0.07299995422363281 sec
5
end of epoch
Time for epoch 7 is 0.07415270805358887 sec
6
end of epoch
Time for epoch 8 is 0.07168102264404297 sec
7
end of epoch
Time for epoch 9 is 0.07899355888366699 sec
8
end of epoch
Time for epoch 10 is 0.08603143692016602 sec
9
end of epoch
Time for epoch 11 is 0.08832859992980957 sec
10
end of epoch
Time for epoch 12 is 0.07544207572937012 sec
11
end of epoch
Time for epoch 13 is 0.07108688354492188 sec
12
end of epoch
Time for epoch 14 is 0.07173371315002441 sec
13
end of epoch
Time for epoch 15 is 0.07080245018005371 sec
14
end of epoch
Time for epoch 16 is 0.07403826713562012 sec
15
end of epoch
Time for epoch 17 is 0.0732719898223877 sec
1

In [ ]:

# DO NOT EDIT
title = "positive_data"
epoch_ckp = EPOCHS - 1
checkpoint_name = checkpoint_dir + "/ckpt__" + str(title) + "__epoch=" + str(epoch_ckp) + "__-" + str(EPOCHS)
# RESTORE CHECKPOINT to variables defined on it
checkpoint.restore(checkpoint_name)

In [ ]:
Z_DIM=4
N_pos = 500
tf.random.set_seed(0)
noise_GEN = tf.random.normal([N_pos, Z_DIM])
# labels_GEN = set for CGAN
data_pos_GEN = generator(noise_GEN, training=False)

In [ ]:
import numpy
proto_tensor = tf.make_tensor_proto(data_pos_GEN)  # convert `tensor a` to a proto tensor
data2 = tf.make_ndarray(proto_tensor) #convert tf tesor to np array
print(data2.shape)


(500, 4)


In [ ]:
import pandas as pd 
pd.DataFrame(data2).to_csv("/content/drive/MyDrive/Privacy/DPGAN_pos_NEWdata.csv")